In [1]:
import pyspark
from pyspark.sql import SparkSession
# Import necessary functions
from pyspark.sql.functions import count as spark_count, desc, col, split, broadcast, lit, rand, concat, floor
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, when
import networkx as nx
import pandas as pd
from pyspark.sql import *
from pyspark.sql.functions import *
import networkx as nx
import pickle
from datetime import date
import numpy as np
from datetime import datetime
import KG_from_parquet as KGF
from vllm import LLM, SamplingParams
import networkx as nx
import pandas as pd
import networkx as nx
import pickle
import numpy as np
import os
from graphframes import GraphFrame
from pyspark.sql import functions as F

/work/shiyxu/myenv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
/work/shiyxu/myenv/lib64/python3.9/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [ ]:
rel_path = "rebel_relations.pkl"
en_map = "mapping.pkl"
entity_path = "rebel_entities.pkl"
# spark initialization
conf = (
    pyspark.SparkConf()
    .setAppName("WikipediaProcessing-SkewDiagnosis") # Changed AppName for clarity
    .setMaster("local[10]")  
    .setAll(
        [
            ("spark.driver.memory", "250g"),
            ("spark.driver.maxResultSize", "32G"), 
            ("spark.memory.fraction", "0.75"),
            ("spark.sql.shuffle.partitions", "2000"), # 8000 may be excessive for just groupBy
            ("spark.driver.memoryOverhead", "8g"),
            ("spark.jars.packages", "graphframes:graphframes:0.8.3-spark3.5-s_2.12")
        ]
    )
)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# initialize llm 
model_path = "../models/Llama-2-13B-chat-hf"
llm = LLM(model_path)
sampling_params = SamplingParams(
    temperature=0.2,  
    max_tokens=50
)
# dics
rel_df, mapping_df, entity_df = KGF.load_dictionaries(spark,rel_path,en_map,entity_path)
# inputs
all_rows = spark.read.parquet("./test_output/graph_data_all.parquet")
# graph
graph_simple_gf = KGF.create_graphframe_from_spark(
    spark, 
    all_rows, 
    mapping_df, 
    rel_df, 
    entity_df, 
    simple=True
)


:: loading settings :: url = jar:file:/work/shiyxu/myenv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/shiyxu/.ivy2/cache
The jars for the packages stored in: /home/shiyxu/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-614bb946-abf0-4527-b094-99031905ab36;1.0
	confs: [default]
	found graphframes#graphframes;0.8.3-spark3.5-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 94ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.3-spark3.5-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

INFO 11-06 22:37:44 __init__.py:207] Automatically detected platform cuda.


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 11-06 22:37:51 config.py:549] This model supports multiple tasks: {'score', 'reward', 'generate', 'embed', 'classify'}. Defaulting to 'generate'.
INFO 11-06 22:37:51 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='../models/Llama-2-13B-chat-hf', speculative_config=None, tokenizer='../models/Llama-2-13B-chat-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=../models/Llama-2-13B-chat-hf, num_sch

...

25/11/06 22:39:25 WARN TaskSetManager: Stage 18 contains a task of very large size (6784 KiB). The maximum recommended task size is 1000 KiB.


filtered_edges 13587287
Creating Vertices DataFrame from nodes_set...


25/11/06 22:39:26 WARN TaskSetManager: Stage 19 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.


filteredv-_edges 2037048
Creating GraphFrame object...


/work/shiyxu/myenv/lib64/python3.9/site-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [6]:
graph_simple_gf.edges.show()

25/10/28 15:57:46 WARN TaskSetManager: Stage 22 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.


+---+--------+------------------+
|src|     dst|       relation_id|
+---+--------+------------------+
|Q31|   Q4916|     Q31_P38_Q4916|
|Q31|Q1061257|Q31_P2852_Q1061257|
|Q31| Q213107| Q31_P1313_Q213107|
|Q31| Q128267|  Q31_P417_Q128267|
|Q31|  Q41614|   Q31_P122_Q41614|
|Q31| Q223933|  Q31_P793_Q223933|
|Q31|Q1160895| Q31_P793_Q1160895|
|Q31|     Q32|      Q31_P530_Q32|
|Q31|     Q38|      Q31_P530_Q38|
|Q31|    Q183|     Q31_P530_Q183|
|Q31|    Q347|     Q31_P530_Q347|
|Q31|    Q408|     Q31_P530_Q408|
|Q31|    Q212|     Q31_P530_Q212|
|Q31|   Q1246|    Q31_P530_Q1246|
|Q31|    Q142|     Q31_P530_Q142|
|Q31|    Q145|     Q31_P530_Q145|
|Q31|     Q16|      Q31_P530_Q16|
|Q31|    Q252|     Q31_P530_Q252|
|Q31|     Q35|      Q31_P530_Q35|
|Q31|     Q43|      Q31_P530_Q43|
+---+--------+------------------+
only showing top 20 rows



In [80]:
def filter_df(graph_simple_gf,current_src ):
    same_src_df = graph_simple_gf.edges.filter(F.col("src") == current_src) # filter same src 

    # concat tables on selected id
    edges_named = (
    same_src_df
    .withColumn("src_name", lookup_name("src"))
    .withColumn("dst_name", lookup_name("dst"))
    .withColumn("prop", extract_property("relation_id"))
    .withColumn("prop_name", lookup_name("prop"))
    .withColumn(
        "relation_name",
        F.concat_ws("", F.concat(F.lit("<subj>"), F.col("src_name")), 
                    F.concat(F.lit("<rel>"), F.col("prop_name")),
                    F.concat(F.lit("<obj>"),F.col("dst_name"))
    )
    )
    )
    start_name = edges_named.select("src_name").first()["src_name"]
    edges_named.show()
    relation_name_set = {row['relation_name'] for row in edges_named.select("relation_name").collect()}
    relations_dict = {
    row['relation_id']: row['relation_name'] 
    for row in edges_named.select("relation_id", "relation_name").collect()
}
    return relations_dict,edges_named

In [86]:
    relations_dict = {
    row['relation_id']: row['relation_name'] 
    for row in edges_named.select("relation_id", "relation_name").collect()
}

25/11/06 21:33:20 WARN TaskSetManager: Stage 260 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 21:33:20 WARN TaskSetManager: Stage 262 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

In [88]:
relations_dict =  {'Q8_P31_Q331769': '<subj>Happiness<rel>instance of<obj>Mood_(psychology)',
 'Q8_P1343_Q1970746': '<subj>Happiness<rel>described by source<obj>Explanatory_Dictionary_of_the_Living_Great_Russian_Language',
 'Q8_P1343_Q867541': '<subj>Happiness<rel>described by source<obj>Encyclopædia_Britannica_Eleventh_Edition',
 'Q8_P460_Q203994': '<subj>Happiness<rel>said to be the same as<obj>Euphoria',
 'Q8_P361_Q13100823': '<subj>Happiness<rel>part of<obj>Quality_of_life',
 'Q8_P1552_Q487': '<subj>Happiness<rel>has quality<obj>Smile',
 'Q8_P1552_Q1117179': '<subj>Happiness<rel>has quality<obj>Life_satisfaction'}

In [ ]:
included_triples=''

In [95]:
def choose_id_LLM(relations_dict, included_triples, llm, sampling_params):
    prompt = f"""You are an AI evaluator. You have chose triples {included_triples} Select only one triple in triples dictionary that is most suitable to continue generating news. Don't repeat included triples. Output the keys as Qx_Px_Qx.
relations:
{relations_dict}
output:"""
    outputs = llm.generate([prompt], sampling_params)
    x = outputs[0].outputs[0].text.strip()
    x=re.findall('Q\d+\_P\d+\_Q\d+',x)
    return x[0]
    # ids = [x.strip() for x in x.split(',') if x.strip()]
    # return ids[0]

key=choose_id_LLM(relations_dict, included_triples, llm, sampling_params)

Processed prompts: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.15s/it, est. speed input: 179.23 toks/s, output: 23.22 toks/s]


In [96]:
relations_dict[key]

'<subj>Happiness<rel>part of<obj>Quality_of_life'

In [71]:
choose_id_LLM(start_node, relation_name_set, included_triples, llm, sampling_params)

['<subj>Happiness<rel>part of<obj>Quality_of_life',
 '<subj>Happiness<rel>part of<obj>Quality_of_life']

In [84]:
start_node="Happiness"
included_triples="<subj>Happiness<rel>part of<obj>Quality_of_life"
relation_name_set= {'<id>P1343<name>described by source',
 '<id>P1552<name>has quality',
 '<id>P31<name>instance of',
 '<id>P361<name>part of',
 '<id>P460<name>said to be the same as'}
prompt = f"""You are an AI evaluator. Select only one triple in triples that is most suitable to generate news for {start_node} based on {included_triples} Don't repeat same triples. Only output the relations id, formatted as pxx. Do NOT add extra information.
relations:
{relation_name_set}
id:"""
outputs = llm.generate([prompt], sampling_params)
x = outputs[0].outputs[0].text.strip()
ids = [x.strip() for x in x.split(',') if x.strip()]
print(ids)

Processed prompts: 100%|████████| 1/1 [00:00<00:00,  1.06it/s, est. speed input: 166.15 toks/s, output: 22.22 toks/s]

['P361\n\nPlease select the most suitable triple to generate news for Happiness.']


In [109]:
same_src_df = graph_simple_gf.edges.filter(F.col("src") == "Q331769") # filter same src = graph_simple_gf.edges.filter(F.col("src") == current_src) # filter same src
if same_src_df.isEmpty():
    print("s")

25/11/06 22:06:41 WARN TaskSetManager: Stage 349 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 22:06:41 WARN TaskSetManager: Stage 350 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.


s


In [ ]:

with open(en_map, "rb") as f:
    id2name = pickle.load(f)
id2name_broadcast = spark.sparkContext.broadcast(id2name)

@F.udf("string")
def lookup_name(qid):
    return id2name_broadcast.value.get(qid, qid)

@F.udf("string")
def extract_property(rel_id):
    parts = rel_id.split('_')
    if len(parts) == 3:
        return parts[1]  # e.g. Q31_P361_Q13116 → P361
    return None

In [ ]:
import re
def filter_df(graph_simple_gf,current_src ):
    same_src_df = graph_simple_gf.edges.filter(F.col("src") == current_src) # filter same src 
    if same_src_df.isEmpty():
        return None, None
    else:
        # concat tables on selected id
        edges_named = (
        same_src_df
        .withColumn("src_name", lookup_name("src"))
        .withColumn("dst_name", lookup_name("dst"))
        .withColumn("prop", extract_property("relation_id"))
        .withColumn("prop_name", lookup_name("prop"))
        .withColumn(
            "relation_name",
            F.concat_ws("", F.concat(F.lit("<subj>"), F.col("src_name")), 
                        F.concat(F.lit("<rel>"), F.col("prop_name")),
                        F.concat(F.lit("<obj>"),F.col("dst_name"))
        )
        )
        )
        start_name = edges_named.select("src_name").first()["src_name"]
        edges_named.show()
        relation_name_set = {row['relation_name'] for row in edges_named.select("relation_name").collect()}
        
        relations_dict = {
        row['relation_id']: row['relation_name'] 
        for row in edges_named.select("relation_id", "relation_name").collect()
    }
        return relations_dict,edges_named
def choose_id_LLM(relations_dict, included_triples, llm, sampling_params):
    prompt = f"""You are an AI evaluator.  Select triples in triples dictionary that are suitable to continue generating news based on {included_triples}. Output the keys as Qx_Px_Qx.
                
                relations:
                {relations_dict}
                
                output:"""
    outputs = llm.generate([prompt], sampling_params)
    print(outputs[0].outputs[0])
    x = outputs[0].outputs[0].text.strip()
    print(x)
    x=re.findall('Q\d+\_P\d+\_Q\d+',x)
    return x[0]
    # ids = [x.strip() for x in x.split(',') if x.strip()]
    # return ids[0]

import random
for start_node in["Q31"]:
    num_triplets = 4
    entities_in_set = [start_node]
    included_triples=[]
    for r in range(2, num_triplets+1):
        current_src = random.choice(list(set(entities_in_set)))
        print(current_src)
        relations_dict,edges_named=filter_df(graph_simple_gf,current_src)
        
        if len(relations_dict)>20:
            all_keys = list(relations_dict.keys())
            random_keys = random.sample(all_keys, 20)
            relations_dict = {key: relations_dict[key] for key in random_keys}
        relations_dict.pop(next_triple_ids,None)
        if relations_dict is not None: 
            try:
                next_triple_ids = choose_id_LLM(relations_dict, included_triples, llm, sampling_params)
                print(next_triple_ids)
                # next_rel_id = re.findall("P\d+", next_rel_id)[0]
                # same_src_rel_def= edges_named.filter(F.lower(F.col("prop"))==next_rel_id.lower())
                # rows_list = same_src_rel_def.select("relation_name").collect()
                # objs=same_src_rel_def.select("dst").collect()
                # string_list = [row['relation_name'] for row in rows_list]
                # objs= [row['dst'] for row in objs]
                # included_triples.extend(string_list)
                # entities_in_set.extend(objs)
                # print(included_triples)
                # print(entities_in_set)
                entities_in_set.extend(re.findall("Q\d+", next_triple_ids))
                included_triples.append(relations_dict[next_triple_ids])
                print(entities_in_set)
                print(included_triples)
            except Exception as e:
                print(e)
        else:
            included_triples.append("none")
            break


Q31


25/11/06 23:41:44 WARN TaskSetManager: Stage 810 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:41:44 WARN TaskSetManager: Stage 811 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:41:45 WARN TaskSetManager: Stage 815 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:41:45 WARN TaskSetManager: Stage 816 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:41:47 WARN TaskSetManager: Stage 818 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:41:47 WARN TaskSetManager: Stage 820 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:41:49 WARN TaskSetManager: Stage 822 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.

+---+--------+------------------+--------+--------------------+-----+--------------------+--------------------+
|src|     dst|       relation_id|src_name|            dst_name| prop|           prop_name|       relation_name|
+---+--------+------------------+--------+--------------------+-----+--------------------+--------------------+
|Q31|   Q4916|     Q31_P38_Q4916| Belgium|                Euro|  P38|            currency|<subj>Belgium<rel...|
|Q31|Q1061257|Q31_P2852_Q1061257| Belgium|112_(emergency_te...|P2852|emergency phone n...|<subj>Belgium<rel...|
|Q31| Q213107| Q31_P1313_Q213107| Belgium|Prime_Minister_of...|P1313|office held by he...|<subj>Belgium<rel...|
|Q31| Q128267|  Q31_P417_Q128267| Belgium|        Saint_Joseph| P417|        patron saint|<subj>Belgium<rel...|
|Q31|  Q41614|   Q31_P122_Q41614| Belgium|Constitutional_mo...| P122|basic form of gov...|<subj>Belgium<rel...|
|Q31| Q223933|  Q31_P793_Q223933| Belgium|  Belgian_Revolution| P793|   significant event|<subj>Belgium<

25/11/06 23:41:49 WARN TaskSetManager: Stage 824 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:41:50 WARN TaskSetManager: Stage 827 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:41:51 WARN TaskSetManager: Stage 828 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
Processed prompts: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.28s/it, est. speed input: 346.95 toks/s, output: 21.90 toks/s]
25/11/06 23:41:55 WARN TaskSetManager: Stage 830 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:41:55 WARN TaskSetManager: Stage 831 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.


CompletionOutput(index=0, text='\n                \n                    Q31_P530_Q668\n                    Q31_P530_Q183\n                    Q31_P832_Q162', token_ids=(13, 462, 13, 462, 1678, 660, 29941, 29896, 29918, 29925, 29945, 29941, 29900, 29918, 29984, 29953, 29953, 29947, 13, 462, 1678, 660, 29941, 29896, 29918, 29925, 29945, 29941, 29900, 29918, 29984, 29896, 29947, 29941, 13, 462, 1678, 660, 29941, 29896, 29918, 29925, 29947, 29941, 29906, 29918, 29984, 29896, 29953, 29906), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)
Q31_P530_Q668
                    Q31_P530_Q183
                    Q31_P832_Q162
Q31_P530_Q668
['Q31', 'Q31', 'Q668']
['<subj>Belgium<rel>diplomatic relation<obj>India']
Q668


...

CompletionOutput(index=0, text='\n                \n                    Q668_P530_Q142\n                    Q668_P530_Q858\n                    Q668_P1365_', token_ids=(13, 462, 13, 462, 1678, 660, 29953, 29953, 29947, 29918, 29925, 29945, 29941, 29900, 29918, 29984, 29896, 29946, 29906, 13, 462, 1678, 660, 29953, 29953, 29947, 29918, 29925, 29945, 29941, 29900, 29918, 29984, 29947, 29945, 29947, 13, 462, 1678, 660, 29953, 29953, 29947, 29918, 29925, 29896, 29941, 29953, 29945, 29918), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)
Q668_P530_Q142
                    Q668_P530_Q858
                    Q668_P1365_
Q668_P530_Q142
['Q31', 'Q31', 'Q668', 'Q668', 'Q142']
['<subj>Belgium<rel>diplomatic relation<obj>India', '<subj>India<rel>diplomatic relation<obj>France']
Q668


25/11/06 23:42:06 WARN TaskSetManager: Stage 852 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:42:07 WARN TaskSetManager: Stage 854 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:42:07 WARN TaskSetManager: Stage 856 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:42:08 WARN TaskSetManager: Stage 859 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:42:09 WARN TaskSetManager: Stage 860 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:42:10 WARN TaskSetManager: Stage 863 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.


+----+--------+-------------------+--------+--------------------+-----+--------------------+--------------------+
| src|     dst|        relation_id|src_name|            dst_name| prop|           prop_name|       relation_name|
+----+--------+-------------------+--------+--------------------+-----+--------------------+--------------------+
|Q668|  Q80524|    Q668_P38_Q80524|   India|        Indian_rupee|  P38|            currency|<subj>India<rel>c...|
|Q668|Q1061257|Q668_P2852_Q1061257|   India|112_(emergency_te...|P2852|emergency phone n...|<subj>India<rel>e...|
|Q668|Q1267849| Q668_P172_Q1267849|   India|  Indo-Aryan_peoples| P172|        ethnic group|<subj>India<rel>e...|
|Q668|  Q69798|   Q668_P172_Q69798|   India|   Dravidian_peoples| P172|        ethnic group|<subj>India<rel>e...|
|Q668| Q192711| Q668_P1313_Q192711|   India|Prime_Minister_of...|P1313|office held by he...|<subj>India<rel>o...|
|Q668| Q512187|  Q668_P122_Q512187|   India|    Federal_republic| P122|basic form of gov

25/11/06 23:42:11 WARN TaskSetManager: Stage 864 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:42:12 WARN TaskSetManager: Stage 866 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 23:42:12 WARN TaskSetManager: Stage 867 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
Processed prompts: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.23s/it, est. speed input: 401.74 toks/s, output: 22.44 toks/s]

CompletionOutput(index=0, text='\n                \n                    Q668_P530_Q774\n                    Q668_P530_Q884\n                    Q668_P2936_', token_ids=(13, 462, 13, 462, 1678, 660, 29953, 29953, 29947, 29918, 29925, 29945, 29941, 29900, 29918, 29984, 29955, 29955, 29946, 13, 462, 1678, 660, 29953, 29953, 29947, 29918, 29925, 29945, 29941, 29900, 29918, 29984, 29947, 29947, 29946, 13, 462, 1678, 660, 29953, 29953, 29947, 29918, 29925, 29906, 29929, 29941, 29953, 29918), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)
Q668_P530_Q774
                    Q668_P530_Q884
                    Q668_P2936_
Q668_P530_Q774
['Q31', 'Q31', 'Q668', 'Q668', 'Q142', 'Q668', 'Q774']
['<subj>Belgium<rel>diplomatic relation<obj>India', '<subj>India<rel>diplomatic relation<obj>France', '<subj>India<rel>diplomatic relation<obj>Guatemala']


In [16]:
entities_in_set

In [102]:
re.findall("Q\d+", next_triple_ids)

['Q8', 'Q331769']

In [103]:
list2=['Q8', 'Q331769']
list1=['Q8']
list1=list1.extend(list2)
print(list1)

None


In [64]:
filter_df(graph_simple_gf,current_src)

25/11/06 20:03:34 WARN TaskSetManager: Stage 118 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 20:03:34 WARN TaskSetManager: Stage 120 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 20:03:35 WARN TaskSetManager: Stage 122 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 20:03:35 WARN TaskSetManager: Stage 124 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

+---------+-------+--------------------+---------------+------------------+----+-----------+--------------------+
|      src|    dst|         relation_id|       src_name|          dst_name|prop|  prop_name|       relation_name|
+---------+-------+--------------------+---------------+------------------+----+-----------+--------------------+
|Q13100823|Q185957|Q13100823_P279_Q1...|Quality_of_life|Quality_(business)|P279|subclass of|<subj>Quality_of_...|
|Q13100823|Q853725|Q13100823_P279_Q8...|Quality_of_life|   Social_relation|P279|subclass of|<subj>Quality_of_...|
|Q13100823|Q151885|Q13100823_P31_Q15...|Quality_of_life|           Concept| P31|instance of|<subj>Quality_of_...|
+---------+-------+--------------------+---------------+------------------+----+-----------+--------------------+



25/11/06 20:03:37 WARN TaskSetManager: Stage 128 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 20:03:37 WARN TaskSetManager: Stage 129 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

('Quality_of_life', {'<id>P279<name>subclass of', '<id>P31<name>instance of'})

In [51]:
import re
next_rel_id
print(re.findall("P\d+", next_rel_id)[0])

P31


In [39]:
same_src_rel_def= edges_named.filter(F.lower(F.col("prop"))==next_rel_id.lower())
rows_list = same_src_rel_def.select("relation_name").collect()
objs=same_src_rel_def.select("dst").collect()
string_list = [row['relation_name'] for row in rows_list]
objs= [row['dst'] for row in objs]
included_triples.extend(string_list)
entities_in_set.extend(objs)

25/11/06 19:44:00 WARN TaskSetManager: Stage 50 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 19:44:01 WARN TaskSetManager: Stage 52 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 19:44:02 WARN TaskSetManager: Stage 54 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 19:44:03 WARN TaskSetManager: Stage 55 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.


In [42]:
current_src = random.choice(list(entities_in_set))
print(current_src)
prop_pairs_set=filter_df(graph_simple_gf,current_src)

Q13100823


25/11/06 19:44:39 WARN TaskSetManager: Stage 59 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 19:44:39 WARN TaskSetManager: Stage 60 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 19:44:40 WARN TaskSetManager: Stage 62 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 19:44:41 WARN TaskSetManager: Stage 64 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

In [6]:
current_src="Q8"
same_src_df = graph_simple_gf.edges.filter(F.col("src") == current_src) # filter same src 

# concat tables on selected id
edges_named = (
same_src_df
.withColumn("src_name", lookup_name("src"))
.withColumn("dst_name", lookup_name("dst"))
.withColumn("prop", extract_property("relation_id"))
.withColumn("prop_name", lookup_name("prop"))
.withColumn(
    "relation_name",
    F.concat_ws("", F.concat(F.lit("<subj>"), F.col("src_name")), 
                F.concat(F.lit("<rel>"), F.col("prop_name")),
                F.concat(F.lit("<obj>"),F.col("dst_name"))
)
)
)
start_name = edges_named.select("src_name").first()["src_name"]


# 2️⃣ 收集 relation_name 列
prop_pairs = edges_named.select("prop", "prop_name") \
    .rdd.map(lambda row: f"<id>{row['prop']}<name>{row['prop_name']}") \
    .collect()
prop_pairs_set = set(prop_pairs)

25/11/06 19:05:48 WARN TaskSetManager: Stage 23 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 19:05:49 WARN TaskSetManager: Stage 24 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 19:05:51 WARN TaskSetManager: Stage 26 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 19:05:51 WARN TaskSetManager: Stage 27 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

In [24]:
def choose_id_LLM(src, prop_pairs_set, included_triples, llm, sampling_params):
    prompt = f"""You are an AI evaluator. Select only one relation in relations that is most suitable to generate news for {src}, with the included triples. Don't repeat same choices in included triples. Only output the relations id, formatted as pxx. Do NOT add extra information.
included triples:{included_triples}
Relations:
{prop_pairs_set}
id:"""
    outputs = llm.generate([prompt], sampling_params)
    x = outputs[0].outputs[0].text.strip()
    ids = [x.strip() for x in x.split(',') if x.strip()]
    return ids[0]

In [13]:
src=current_src

In [7]:
prop_pairs_set

{'<id>P1343<name>described by source',
 '<id>P1552<name>has quality',
 '<id>P31<name>instance of',
 '<id>P361<name>part of',
 '<id>P460<name>said to be the same as'}

In [17]:
included_triples=string_list

In [25]:
print(choose_id_LLM(src, prop_pairs_set,included_triples, llm, sampling_params))

Processed prompts: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.10it/s, est. speed input: 155.66 toks/s, output: 22.08 toks/s]

P361

Please select the most suitable relation to generate news for Q8.


In [26]:
same_src_rel_def= edges_named.filter(F.lower(F.col("prop"))=="p361")
rows_list = same_src_rel_def.select("relation_name").collect()
string_list = [row['relation_name'] for row in rows_list]

25/11/06 19:24:32 WARN TaskSetManager: Stage 31 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
25/11/06 19:24:33 WARN TaskSetManager: Stage 32 contains a task of very large size (2608 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

In [27]:
string_list

['<subj>Happiness<rel>part of<obj>Quality_of_life']